<a href="https://colab.research.google.com/github/felipe-parodi/DL4DataScience/blob/main/Week10_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CIS Week 10 Homework

**Instructor:** Lyle Ungar

**Content Creators:** Sanjeevini Ganni, Brittany Shields, Alessandra Rossi

In [ ]:
#@markdown What is your Pennkey and pod? (text, not numbers, e.g. bfranklin)
my_pennkey = 'fparodi' #@param {type:"string"}
my_pod = 'superfluous-lyrebird' #@param ['Select', 'euclidean-wombat', 'sublime-newt', 'buoyant-unicorn', 'lackadaisical-manatee','indelible-stingray','superfluous-lyrebird','discreet-reindeer','quizzical-goldfish','ubiquitous-cheetah','nonchalant-crocodile','fashionable-lemur','spiffy-eagle','electric-emu','quotidian-lion','astute-jellyfish', 'quantum-herring']

# start timing
import time
try:t0;
except NameError: t0 = time.time()

We **strongly** recommend that you keep a separate document offline with your answers, and paste them in when you're ready to submit. Colab may reset and clear your notebook after a period of inactivity.

Go to Runtime -> Change runtime type -> Set Hardware Accelerator to "GPU"

###**Learning Objectives**
Finetune a Transformer model

How might word embedding perpetuate systemic bias?

How might we work with policymakers and companies to reduce the spread of fake news?

## Part I: Finetune GPT-2





GPT2 is a large-scale transformer-based language model developed by OpenAI. It is pre-trained on a large corpus of dataset of 8 million web pages has around 1.5 billion parameters. GPT2 is useful for language generation tasks, that is it predicts the next word, given some text.  

In this excersice we will be finetuning DistillGPT-2, distilled version of GPT2. We want the language model to generate wikipedia style of text. Hence, we will finetune distillGPT on [wikitext-2-raw-v1](https://huggingface.co/datasets/wikitext) dataset scrapped from wikipedia articles.

In [ ]:
#@title Install
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.0MB 18.7MB/s 
     |████████████████████████████████| 870kB 50.0MB/s 
     |████████████████████████████████| 3.2MB 47.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=b8a6a49abdbe7ebdea6d7a4c82cbf145493e3aca51215a6de35547034e7ab6a1
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
     |████████████████████████████████| 194kB 25.3MB/s 
     |████████████████████████████████| 245kB 47.1MB/s 
     |████████████████████████████████| 112kB 20.9MB/s 


In [ ]:
#@title Imports
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments

In [ ]:
#@title Load Datset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91. Subsequent calls will reuse this data.


Example of Dataset:

In [ ]:
datasets["train"][48]

{'text': " Two manga adaptations were produced , following each of the game 's main female protagonists Imca and Riela . They were Senjō no Valkyria 3 : Namo naki Chikai no Hana ( 戦場のヴァルキュリア3 名もなき誓いの花 , lit . Valkyria of the Battlefield 3 : The Flower of the Nameless Oath ) , illustrated by Naoyuki Fujisawa and eventually released in two volumes after being serialized in Dengeki Maoh between 2011 and 2012 ; and Senjō no Valkyria 3 : -Akaki Unmei no Ikusa Otome- ( 戦場のヴァルキュリア3 -赤き運命の戦乙女- , lit . Valkyria of the Battlefield 3 -The Valkyrie of the Crimson Fate ) , illustrated by Mizuki Tsuge and eventually released in a single volume by Kadokawa Shoten in 2012 . \n"}

####Tokenizer

In [ ]:
#Define the model Checkpoint
model_checkpoint = "distilgpt2"

#Tokenizer  
#To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. 
#This is all done by the AutoTokenizer class:  
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

#We define a function to call the tokenizer on our texts:
def tokenize_function(examples):
    return tokenizer(examples["text"])

#Apply tokenizer to all the splits in our datasets object and drop the text coolumn
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [ ]:
#we need to concatenate all our texts together then split the result in small chunks of a certain block_size
#Define the block size for training
block_size = 128

#preprocessing function that will group our texts
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

###Load pre-trained model

In [ ]:
#####Pretrained model
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

###Run Generations before Finetuning

In [ ]:
def run_generations(PROMPT_TEXT):
  device = model.device
  input_ids = tokenizer.encode(PROMPT_TEXT, return_tensors="pt").to(device)

  sample_output = model.generate(
      input_ids, 
      do_sample=True, 
      max_length=100, 
      top_p=0.95,
      num_return_sequences=3, 
      early_stopping=True
  )


  for i, output in enumerate(sample_output):
    print("Generation: "+ str(i) +"\n" + 100 * '-')
    print(tokenizer.decode(output, skip_special_tokens=True))
    print()


########
PROMPT_TEXT = 'Fast and the Furious '
run_generations(PROMPT_TEXT)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation: 0
----------------------------------------------------------------------------------------------------
Fast and the Furious????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????

Generation: 1
----------------------------------------------------------------------------------------------------
Fast and the Furious!!!
The second episode of the show was the finale episode of their one-episode, and it was originally titled "The New Order". This is a continuation of the series finale episode of Episode 9 - the same thing the previous episode would have had.
At the time of filming, the series had an active presence at the beginning of the first episode. Due to the filming of the episodes (as is the case with the first season), that was done in the form

Generation: 2
------------------------------------------------------------------------------

In [ ]:
############## Run Generation for different prompt texts of your choice
PROMPT_TEXT = 'Monkey'
run_generations(PROMPT_TEXT)


PROMPT_TEXT = 'Computer Vision'
run_generations(PROMPT_TEXT)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation: 0
----------------------------------------------------------------------------------------------------
Monkey: *Ajax> *Jax> *RJax> *Djax> *TJax> *RJax> *TJax> *Djax> *RJax> *Djax> *TJax> *Djax> *TJax> *RJax> *Djax> *Djax> *TJax> *Djax> *Djax> *RJ

Generation: 1
----------------------------------------------------------------------------------------------------
Monkey.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key.key

Generation: 2
----------------------------------------------------------------------------------------------------
Monkey_3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3

Generation: 0
----------------------------------------------------------------------------------------------------
Computer Vision (vTvV) is a vision, which takes control of the light to produce infor

###Fine-Tuning (This will take around 20-25 minutes)

In [ ]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=2,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,3.757000,3.670282,7.801400,247.520000
2,3.668200,3.654916,7.788700,247.924000


TrainOutput(global_step=4668, training_loss=3.752927543162891, metrics={'train_runtime': 565.0251, 'train_samples_per_second': 8.262, 'total_flos': 2348513500594176.0, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 350532, 'init_mem_gpu_alloc_delta': 334744576, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 845257, 'train_mem_gpu_alloc_delta': 995278336, 'train_mem_cpu_peaked_delta': 155864427, 'train_mem_gpu_peaked_delta': 1491477504})

Perplexity of Validation Set

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 38.66


###Run Generations after Finetuning
Run generations for different prompts and compare them with the initial generations.

In [ ]:
PROMPT_TEXT = 'Fast and the Furious '
run_generations(PROMPT_TEXT)

############## Run Generation for different prompt texts of your choice
PROMPT_TEXT = 'Bias'
run_generations(PROMPT_TEXT)


PROMPT_TEXT = 'Monkey'
run_generations(PROMPT_TEXT)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation: 0
----------------------------------------------------------------------------------------------------
Fast and the Furious 
 = = = Release = = = 
 On November 18, 2012, they had an exclusive exclusive exclusive deal with the production company that brought in their first full production unit. On November 23, 2012, the original version of the title was released on Blu @-@ray in Japan, and the official release on Blu @-@ray in the United States on November 18, 2012 was released exclusively on Blu @-@ray in the United Kingdom. The initial release

Generation: 1
----------------------------------------------------------------------------------------------------
Fast and the Furious 
 = = Release = = 
 In 2008, The Mortal Kombat was announced, and in April 2009, was released as a digital download. At the time, it was the second game of the 2006 reboot, and the second game to be released since 2003. It took the franchise to the next level, and released the game in late September

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation: 0
----------------------------------------------------------------------------------------------------
Biasi and Orakon ( two of whom were members of Orakon and Orakon ), both were sent out as soldiers (, with Orakon and Orakon ) and in January 1943, they participated in the liberation of the capital city of Ramadi. As a result, at 1 : 22, the army left Ramadi under the command of General Anwar Sadu and marched into Ramadi and was ordered to withdraw immediately to Ramadi and capture the city of

Generation: 1
----------------------------------------------------------------------------------------------------
Bias in her new book The Other Way, argues that, among contemporary literature, the literary critic Paul E. Gere argues that the way in which writers were " the key to the meaning of the word in American literature ( such as the early American's ). This view is not an endorsement of the view of the writers — it might be argued that a sense of kinship existed in the nov

In [ ]:
#@markdown Question 0: How do the generations after finetuning compare to the generations before finetuning. Do you think finetuning acheived its purpose of generating Wikipedia like texts? 
q0 = "The generations after finetuning are better in that the sentences make grammatical sense? But they rarely relate to the original word. Finetuning achieved its purpose of generating wikipedia like texts " #@param{type:"string"}

try:t0;
except NameError: t0 = time.time()

## Part II: Primer on Gender Bias in Word Embeddings

Read the following primer that describes the societal implications of gender bias in word embeddings: \\
“Man is to Doctor as Woman is to Nurse: The Gender Bias of Word Embeddings: Why we should worry about gender inequality in Natural Language Processing techniques” \\
https://towardsdatascience.com/gender-bias-word-embeddings-76d9806a0e17 \\
*9 min read*, Tommaso Buonocore, 2019


In [ ]:
#@markdown Question 1: Identify a compelling quote or example from the article that resonated with you.  How does it emphasize the societal responsibility of data scientists?
q1 = "One compelling quote that resonated with me is \"language itself is designed to reflect social asymmetries.\" This quote resonated with me because I have long appreciated how important language is in expressing our values to one another, and, I think humans do a pretty terrible job at it. We are very bad at \"precise communication,\" and we oft don't communicate what we truly feel and think. I have always been pedantic on using precise language for this exact reason: if we don't standardize how we speak, not only will we never truly express ourselves to another individual, but we will also continue to linguistically perpetuate the \"asymmetries\" (read: harmful biases) of society." #@param{type:"string"}

try:t1;
except NameError: t1 = time.time()

##Part III: Tech Paper on Gender Bias in Word Embeddings

In the previous article, the author referenced that data scientists are introducing new strategies to reduce biases in word embeddings.  In 2016, researchers from Boston University and Microsoft Research published a paper with significant findings demonstrating the prevalence of gender bias in word embeddings, as well as proposed mitigation measures.  A 5-minute summary of the technical components of the paper can be found [here](https://towardsdatascience.com/tackling-gender-bias-in-word-embeddings-c965f4076a10) and the original paper can be found [here](https://arxiv.org/abs/1607.06520). \

“Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings” \
https://arxiv.org/abs/1607.06520 \
*30 min read*, 2016





In [ ]:
#@markdown Question 2: In the “Discussion” section, the authors consider the responsibility of machine language programmers with regards to bias in word embeddings.  Describe their stance.
q2 = "The stance of the authors with regards to the responsibility of machine language programmers is to indeed take the approach of debiasing algorithms to combat bias in word embeddings. It is better to err on the side of caution than to risk perpetuating further harm." #@param{type:"string"}

#@markdown Question 3: How could word embeddings amplify societal stereotypes?
q3 = "As we've seen, some embeddings can be damaging: male-doctor, female-nurse; computer programmer-he, housekeeper-she. Such embeddings reinforce a toxic, harmful, patriarchal bearing on society when such algorithms are deployed like Google's suggested search feature (e.g., \"should women... {suggested: stay home}\"). By debiasing these algorithms, (i.e., removing gender from the embedding), we can prevent machine language algorithms from inadvertently amplifying harmful stereotypes." #@param{type:"string"}

#@markdown Question 4: How could the authors’ proposed technological interventions address this concern?
q4 = "Removing gender from the embedding to equate particular directions of a vector, such as the example of \"grandmother/grandfather\" distance to \"babysit\" can prevent the amplification of societal stereotypes by altogether removing the potential source of bias, gender." #@param{type:"string"}

try:t2;
except NameError: t2 = time.time()

##Part IV: Fake News

The following article provides an overview of neural fake news, including potential defenses against it as well as suggestions for further research.

“An Exhaustive Guide to Detecting and Fighting Neural Fake News using NLP” \
•	https://www.analyticsvidhya.com/blog/2019/12/detect-fight-neural-fake-news-nlp/ \
•	*16-minute read*, Analytics Vidhya, 2019

In 2019, Jack Clark, the Policy Director at OpenAI, gave testimony to the US House Intelligence Committee.  You can either read his written 10-page testimony here or watch his live testimony here (jump to minute 11:00 to watch his 5 minute testimony).

Written Testimony of Jack Clark, Policy Director, OpenAI \
•	https://intelligence.house.gov/uploadedfiles/clark_deepfakes_sfr.pdf \
•	or \
•	https://www.c-span.org/video/?461679-1/house-intelligence-committee-hearing-deepfake-videos \
*Jump to minute 11:00* \
•	House Permanent Select Committee on Intelligence, 2019 \


In [ ]:
#@markdown Question 5: Jack Clark enumerated several ideas for interventions.  Discuss one of the specific ideas he suggested.
q5 = "One specific idea Jack Clark suggested to combat fake news is comprehensive AI education. I had suggested this idea in the computer vision homework a while back. A good way to identify fake news, especially DeepFakes, is if everyone is knowledgeable about spotting it. In essence, we could all benefit from a healthy dose of skepticism. " #@param{type:"string"}

#@markdown Question 6: How might technologists work with policymakers and companies to prevent the spread of fake news?
q6 = "Technologists and large social media platforms can work with policymakers to prevent the spread of fake news at a myriad of levels. The biggest challenge is overcoming the barrier of mass media. Today fake news spreads like wildfire on social media platforms on televised news networks like CNN and Fox News, and smaller, more local platforms. Technologists can work with policy makers to set up teams regulating information at distinct levels: local, state, and federal media dissemination. However, it is possible that this could bring us closer to a surveillance state." #@param{type:"string"}

#@markdown Question 7: •	OpenAI is not making GPT-3 publicly available, claiming that they are concerned that it might be used for doing bad. Do you think that was a good decision? Why?
q7 = "I think it is a wise decision by OpenAI leaders. Every sort of technology can be wielded negatively, and the OpenAI folks were right in acting preemptively before some malicious agent applied GPT-3 to generate and disseminate fake news or stir chaos (think Russian media bots during the 2016/2020 election cycles)." #@param{type:"string"}

try:t3;
except NameError: t3 = time.time()

## PART V: Know your Pod

With two other members of your pod, do the following. Have each of them recommend you their favorite song. Then, listen to those songs and write down your thoughts for each. (~100 words each)

In [ ]:
know_a_pod_1 = "Joseph Goodman gave me the song Dynasty by MIIA (Elephante remix). I really enjoyed this song. I'm a big fan of pop songs, and the House remix reminds me of high school/college? I feel like this was really big (and I'm sure it still is). The lyrics are pretty sad though: \"and all I gave you is gone... the scar I can't reverse.\" I hope Joseph is ok. The song's great though, closer to my tastes than After Life." #@param{type:"string"}
know_a_pod_2 = "Nicole Chiou gave me the song After Life by Tchami. The song is great, really. I don't typically listen to House music (and its varieties), but I've also never given it a chance. I just always found it... repetitive. The drop in this song is great though, and the song will get you moving (dancing as I type this). " #@param{type:"string"}

try:t4;
except NameError: t4 = time.time()

#Submission

Once you're done, click on 'Share' and add the link to the box below.

In [ ]:
link = 'https://colab.research.google.com/drive/14x4k1rcNPP9PEvfJuL2btPcywSqqQa-K?usp=sharing' #@param {type:"string"}

In [ ]:
import time
import numpy as np
import urllib.parse
from IPython import display
from IPython.display import IFrame


t5 = time.time()

#@markdown #Run Cell to Show Airtable Form
#@markdown ##**Confirm your answers and then click "Submit"**


def prefill_form(src, fields: dict):
  '''
  src: the original src url to embed the form
  fields: a dictionary of field:value pairs,
  e.g. {"pennkey": my_pennkey, "location": my_location}
  '''
  prefill_fields = {}
  for key in fields:
      new_key = 'prefill_' + key
      prefill_fields[new_key] = fields[key]
  prefills = urllib.parse.urlencode(prefill_fields)
  src = src + prefills
  return src



#autofill fields if they are not present
#a missing pennkey and pod will result in an Airtable warning
#which is easily fixed user-side.
try: my_pennkey;
except NameError: my_pennkey = ""

try: my_pod;
except NameError: my_pod = "Select"

try: q0;
except NameError: q0 = ""

try: q1;
except NameError: q1 = ""

try: q2;
except NameError: q2 = ""

try: q3;
except NameError: q3 = ""

try: q4;
except NameError: q4 = ""

try: q5;
except NameError: q5 = ""

try: q6;
except NameError: q6 = ""

try: q7;
except NameError: q7 = ""

try: know_a_pod_1;
except NameError: know_a_pod_1 = ""

try: know_a_pod_2;
except NameError: know_a_pod_2 = ""

try: link;
except NameError: link = ""

fields = {"pennkey": my_pennkey,
          "pod": my_pod,
          "q0": q0,
          "q1": q1,
          "q2": q2,
          "q3": q3,
          "q4": q4,
          "q5": q5,
          "q6": q6,
          "q7": q7,
          "know_a_pod_1": know_a_pod_1,
          "know_a_pod_2": know_a_pod_2,
          "link": link}

src = "https://airtable.com/embed/shrgundiXUEJFVzwd?"


#now instead of the original source url, we do: src = prefill_form(src, fields)
display.display(IFrame(src = prefill_form(src, fields), width = 800, height = 400))